In [23]:
import pandas as pd
import logging
from os import path
import mysql.connector
import datetime as dt
from tensorflow import keras
import json

In [7]:
logging.basicConfig(filename='logging/app.log', filemode='w', format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')

In [11]:
#data.py
def load_df(stock_symbol):
    """
    Return dataframe from histdailyprice3
    """
    HOST="143.244.188.157"
    PORT="3306"
    USER="patrick-finProj"
    PASSWORD="Pat#21$rick"

    dpath = f"histdailyprice3/{stock_symbol}.csv"
    if path.isfile(dpath):
        load_csv(dpath)
    else:
        logging.info(f'Load data from MySQL.')
        try: 
            conn = mysql.connector.connect(
                host=HOST,
                port=PORT,
                user=USER,
                password=PASSWORD,
                database="GlobalMarketData"
            )
            query = f"SELECT Date, Exchange, Close, Open, High, Low, Volume from histdailyprice3 WHERE Symbol='{stock_symbol}';"
            histdailyprice3 = pd.read_sql(query, conn)
            conn.close()
            df = histdailyprice3.copy()
            df.to_csv(dpath, index=False)
            return df
        except Exception as e:
            logging.error("Exception occurred", exc_info=True)

In [25]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import logging

# Tune MLP
def tune(X, y):
    logging.info(f'Tune MLP hyperparameters')
    try:
        param_grid = {'hidden_layer_sizes': [(100, 50), (50, 50), (10, 100)],
          'activation': ['relu','tanh','logistic'],
          'alpha': [0.0001, 0.05, 0.0005, 0.00005],
          'learning_rate': ['constant','adaptive'],
          'solver': ['adam']}
        mlp = MLPRegressor()
        clf = GridSearchCV(mlp, param_grid, n_jobs=-1)
        clf.fit(X, y)
        top_params = clf.best_params
        logging.info(f'Best MLP parameters {top_params}')

        return top_params
    except Exception as e:
        logging.error("Exception occurred", exc_info=True)

# Predict with MLP using best parameters
def predict(X, y, params):
    try:
        clf = MLPRegressor(params)
        clf.fit(X.iloc[:-1].values, y[1:].values.reshape(-1,))
        prediction = clf.predict(X.iloc[-1:])
        return prediction
    except Exception as e:
        logging.error("Exception occurred", exc_info=True)

In [20]:
def run_main():
    logging.info(f'Start main.py')
    try:
        symbol_list = ['SPY']
        logging.info(f'Loop through symbols')
        for symbol in symbol_list:
            logging.info(f'Generate predictions for {symbol}')
            try:
                df = load_df(symbol)
                exchange = df.Exchange.iloc[0]

                # Data Processing
                logging.info(f'Data processing for {symbol}')
                close = df["Close"]
                returns = returns(close)
                X, realized_vol = realized_vol(returns, rolling_window=5)

                output_dict = {
                    "Date": date.now(),
                    "Symbol": symbol,
                    "Exchange": exchange,
                    #"garch": garch_predict(symbol, returns),
                    #"svr": svr_predict(symbol, X, realized_vol),
                    #"mlp": mlp_predict(symbol, X, realized_vol),
                    #"LSTM": lstm_predict(symbol, close)
                }
                output_df = pd.DataFrame(output_dict)
                output_df.to_csv(f'output/{symbol}_{date.now()}.csv')
                print(output_df)
            except Exception as e:
                logging.error("Exception occurred", exc_info=True)
    except Exception as e:
        logging.error("Exception occurred", exc_info=True)

In [29]:
stock_symbol = "SPY"

HOST="143.244.188.157"
PORT="3306"
USER="patrick-finProj"
PASSWORD="Pat#21$rick"

try: 
    conn = mysql.connector.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database="GlobalMarketData"
    )
    query = f"SELECT Date, Exchange, Close, Open, High, Low, Volume from histdailyprice3 WHERE Symbol='{stock_symbol}';"
    histdailyprice3 = pd.read_sql(query, conn)
    conn.close()
    df = histdailyprice3.copy()
except Exception as e:
    logging.error("Exception occurred", exc_info=True)

In [37]:
df.head()

,Date,Exchange,Close,Open,High,Low,Volume
0,1998-01-01,AMEX,97.0625,97.0625,97.0625,97.0625,0
1,1998-01-02,AMEX,97.3125,97.6563,96.5313,97.5625,2333800
2,1998-01-05,AMEX,97.7813,98.4375,96.7813,97.7813,4127800
3,1998-01-06,AMEX,97.2500,97.2813,96.1875,96.2188,3154900
4,1998-01-07,AMEX,96.0625,96.7188,95.2188,96.4688,4424200


In [49]:
# processing.py
# Generate returns for Volatiltiy models
def get_returns(close, split=0.01):
    try:
        logging.info(f'Generate returns')
        returns = 100 * close.pct_change().dropna()
        n = int(len(returns)*split)
        split_date = returns[-n:].index
        return returns, n, split_date
    except Exception as e:
        logging.error("Exception occurred", exc_info=True)

# Compute realized volatility
def get_realized_vol(returns, rolling_window):
    try:
        logging.info(f'Generate X and realized_vol with rolling window {rolling_window}')
        realized_vol = returns.rolling(rolling_window).std()
        returns_svm = returns ** 2
        X = pd.concat([realized_vol, returns_svm], axis=1, ignore_index=True)
        X = X[rolling_window-1:].copy()
        X.reset_index(drop=True, inplace=True)
        realized_vol.dropna(inplace=True)
        return X, realized_vol
    except Exception as e:
        logging.error("Exception occurred", exc_info=True)

In [50]:
close = df["Close"]
returns, n, split_date = get_returns(close)
X, realized_vol = get_realized_vol(returns, rolling_window=5)

In [51]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import logging

# Tune MLP
def tune(X, y):
    logging.info(f'Tune MLP hyperparameters')
    try:
        param_grid = {'hidden_layer_sizes': [(100, 50), (50, 50), (10, 100)],
          'activation': ['relu','tanh','logistic'],
          'alpha': [0.0001, 0.05, 0.0005, 0.00005],
          'learning_rate': ['constant','adaptive'],
          'solver': ['adam']}
        mlp = MLPRegressor()
        clf = GridSearchCV(mlp, param_grid, n_jobs=-1)
        clf.fit(X, y)
        top_params = clf.best_params
        logging.info(f'Best MLP parameters {top_params}')

        return top_params
    except Exception as e:
        logging.error("Exception occurred", exc_info=True)

# Predict with MLP using best parameters
def predict(X, y, params):
    try:
        clf = MLPRegressor(params)
        clf.fit(X.iloc[:-1].values, y[1:].values.reshape(-1,))
        prediction = clf.predict(X.iloc[-1:])
        return prediction
    except Exception as e:
        logging.error("Exception occurred", exc_info=True)

In [52]:
def mlp_predict(symbol, X, realized_vol):
    try:
        dpath = f"model/params/mlp/{symbol}.csv"
        if path.isfile(dpath):
            logging.info(f'Load params from {dpath}')
            with open(dpath, "r") as ifile:
                params = json.load(ifile)
        else:
            params = mlp.tune(X, realized_vol)
            with open(dpath, "w") as outfile:
                json.dump(params, outfile)
            logging.info(f'Export best MLP parameters to {dpath}')           
        mlp_predict = mlp.predict(X, realized_vol, params)
        return mlp_predict
    except Exception as e:
        logging.error("Exception occurred", exc_info=True)

In [55]:
param_grid = {'hidden_layer_sizes': [(100, 50), (50, 50), (10, 100)],
          'activation': ['relu','tanh','logistic'],
          'alpha': [0.0001, 0.05, 0.0005, 0.00005],
          'learning_rate': ['constant','adaptive'],
          'solver': ['adam']}
mlp = MLPRegressor()
clf = GridSearchCV(mlp, param_grid, n_jobs=-1)
clf.fit(X, realized_vol)
top_params = clf.best_params_
logging.info(f'Best MLP parameters {top_params}')

AttributeError: 'GridSearchCV' object has no attribute 'best_params'

In [54]:
params